<a href="https://colab.research.google.com/github/Amal-Baby-Mathews/Amal_TensorFlow2.0/blob/main/LSTM%7CShap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install shap
!pip install pynput

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 3.6 MB/s eta 0:00:00
  Created wheel for evdev: filename=evdev-1.7.0-cp310-cp310-linux_x86_64.whl size=92530 sha256=67219767ee92d45069a29af4a64ebd268b35dc482729d7b1ed7283b128834ca8
  Stored in directory: /root/.cache/pip/wheels/16/c9/19/2c0af606c704d962276592b03175d1796077199329e2777ec0
Successfully built evdev


In [6]:
import numpy as np
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Reshape
from tensorflow.keras.callbacks import EarlyStopping
from shap import KernelExplainer, summary_plot,sample

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
data = pd.read_csv('/content/drive/MyDrive/Data_ML/temperory_dataset2.csv')
data = data.dropna()
X = data.drop('be_system_abort_shutdown', axis=1)
y = data['be_system_abort_shutdown']

In [9]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# Reshape the input data for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))


In [10]:
print(X.head(3))
print(y.head(3))

       be_ambient_tt_scaled_value  be_scg_n2  be_stream2methane  \
11708                   19.379999   1.198203          97.400002   
11709                   19.379999   1.198203          97.400002   
11710                   19.379999   1.198203          97.400002   

       al_productgasoutlet_temp  pdc_powermeterphasenuetral_amps  
11708                 50.312500                         0.257775  
11709                 50.296875                         0.257775  
11710                 50.312500                         0.257777  
11708    1
11709    1
11710    1
Name: be_system_abort_shutdown, dtype: int64


In [13]:
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[2])))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=5)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, callbacks=[early_stop])


14404/14404 [==============================] - 30s 2ms/step - loss: 0.0051 - accuracy: 0.9992 - val_loss: 1.4440e-10 - val_accuracy: 1.0000


In [14]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')


3601/3601 [==============================] - 5s 1ms/step - loss: 1.4440e-10 - accuracy: 1.0000
Test accuracy: 1.0


In [15]:
#X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
explainer = KernelExplainer(model.predict, data=X_train)



14404/14404 [==============================] - 17s 1ms/step


In [16]:
model_lstm = Sequential()
model_lstm.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(Dense(1, activation='sigmoid'))

# Create a separate model for SHAP
input_layer = Input(shape=(X_train.shape[1] * X_train.shape[2],))
reshaped = Reshape((X_train.shape[1], X_train.shape[2]))(input_layer)
output = model_lstm(reshaped)
model_shap = Model(inputs=input_layer, outputs=output)

# Compile and train the model
# ... (code omitted for brevity)

# Prepare the data for feature importance analysis
background_data = X_train.reshape(-1, X_train.shape[1] * X_train.shape[2])
background_data_summary = sample(background_data, 1)  # Use 1000 representative samples
explainer = KernelExplainer(model_shap.predict, background_data_summary)

# Downsample X_test to 5000 samples
X_test_downsampled = resample(X_test.reshape(-1, X_test.shape[1] * X_test.shape[2]),
                              replace=False,
                              n_samples=500,
                              random_state=42)

# Calculate feature importance using SHAP
shap_values = explainer.shap_values(X_test_downsampled)
#summary_plot(shap_values, X.columns)


1/1 [==============================] - 0s 312ms/step


  0%|          | 0/500 [00:00<?, ?it/s]

1/1 [==============================] - 0s 13ms/step


In [17]:
def predict_failure(data):
    data_scaled = scaler.transform(data)
    data_reshaped = data_scaled.reshape((-1, 1,data_scaled.shape[1]))
    prediction = model.predict(data_reshaped)
    failure_probability = prediction[0][0]
    return failure_probability

# Set a threshold for failure prediction
threshold = 0.7

In [18]:
print(X_test[0:10].reshape(10, -1))

[[0.54822324 0.15074714 0.4716992  0.77862595 0.93951989]
 [0.55076137 0.15074714 0.4716992  0.77902772 0.93950373]
 [0.54822324 0.15074714 0.4716992  0.77902772 0.93949952]
 [0.55076137 0.15074714 0.4716992  0.77862595 0.93949453]
 [0.54822324 0.15074714 0.4716992  0.77862595 0.93948684]
 [0.55076137 0.15074714 0.4716992  0.77862595 0.93517782]
 [0.55076137 0.15074714 0.4716992  0.77862595 0.93521002]
 [0.55076137 0.15074714 0.4716992  0.77862595 0.93521002]
 [0.55076137 0.15074714 0.4716992  0.77862595 0.93952522]
 [0.54822324 0.15074714 0.4716992  0.77902772 0.93953001]]


In [19]:
def get_latest_system_data(window_size=5):
    # Initialize a counter to keep track of the current window
    window_counter = 0
    num_features = X_test.shape[2]  # Get the number of features from X_test

    while True:
        # Calculate the start and end indices for the current window
        start = window_counter * window_size
        end = start + window_size

        # Check if we have reached the end of X_test
        if end > X_test.shape[0]:
            # Reset the counter to start from the beginning
            window_counter = 0
            start = 0
            end = window_size
            return None
        # Get the current window of data
        current_window = X_test[start:end]

        # Reshape the window to match the expected input shape
        latest_data = current_window.reshape(window_size, 5)

        # Simulate new data by adding random noise
        noise = np.random.normal(0, 0.1, size=latest_data.shape)
        latest_data += noise

        # Increment the window counter
        window_counter += 1
        print(latest_data)
        yield latest_data

In [20]:
print(shap_values)

[[[-1.51746472e-03]
  [-5.26150068e-04]
  [ 6.69678052e-04]
  [-1.46478415e-05]
  [ 7.41866728e-03]]

 [[ 3.32978169e-03]
  [-3.11019023e-04]
  [-7.74539510e-04]
  [ 1.04735295e-05]
  [ 7.33420054e-03]]

 [[-3.82762154e-04]
  [-3.41500839e-04]
  [-1.46887799e-03]
  [-2.61326631e-05]
  [ 7.35075672e-03]]

 ...

 [[-5.65847953e-03]
  [-3.65696351e-04]
  [-9.18976466e-04]
  [-7.87844261e-05]
  [ 7.33909309e-03]]

 [[ 8.21071863e-04]
  [-3.67097060e-04]
  [-1.57432954e-03]
  [ 2.07751989e-05]
  [ 7.18163749e-03]]

 [[-3.49068244e-03]
  [-2.44956215e-04]
  [-1.49137874e-03]
  [-2.27948030e-05]
  [ 7.21450051e-03]]]


In [21]:
data_generator = get_latest_system_data(window_size=5)  # Set the desired window size


while True:
  # Get the latest system data
  latest_data = next(data_generator)

  # Predict failure probability
  failure_prob = predict_failure(latest_data)

  if failure_prob > threshold:
      print(f'Potential system failure detected! Probability: {failure_prob:.2f}')
      print('Field variations contributing to the failure:')
      for feature, importance_list in zip(X.columns, shap_values[0].tolist()):
            for importance in importance_list:
                if importance > 0:
                    if abs(importance) > 0.001:  # Print only if the absolute SHAP value is greater than 0.001
                      print(f'- {feature}: Importance = {"{}".format(importance)}')
  else:
      print('System operating normally.')

[[0.49097433 0.190824   0.35897929 0.77041249 0.80180709]
 [0.56168582 0.10104754 0.38957872 0.80060306 0.73401688]
 [0.45544475 0.09284715 0.35718405 0.75793482 0.9713253 ]
 [0.71439283 0.21652851 0.46364162 0.87743837 1.02744282]
 [0.65070807 0.14385025 0.588622   0.88568048 0.8880533 ]]
1/1 [==============================] - 0s 16ms/step
System operating normally.
[[ 6.96126337e-01  3.91606344e-04  3.69976398e-01  6.69242147e-01
   7.99895547e-01]
 [ 5.15755752e-01 -7.75776380e-02  5.72756087e-01  8.35756261e-01
   8.94146430e-01]
 [ 4.29003372e-01  3.82030102e-01  5.02790755e-01  7.27495105e-01
   1.07836986e+00]
 [ 4.85298722e-01  2.79472440e-02  3.63104484e-01  8.15940332e-01
   9.11211291e-01]
 [ 4.25864788e-01  2.82242413e-01  4.89672962e-01  6.00633558e-01
   9.23109720e-01]]
1/1 [==============================] - 0s 17ms/step
System operating normally.
[[0.66848693 0.09045232 0.44058995 0.79338012 0.82762684]
 [0.789487   0.28507317 0.56613384 0.64027283 0.95180493]
 [0.37059

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 14ms/step
System operating normally.
[[0.56605006 0.3166839  0.59450153 0.58488182 0.86156724]
 [0.80283484 0.02380961 0.64840234 0.90464713 0.87476692]
 [0.56568418 0.26852132 0.46067881 0.82818933 0.93110072]
 [0.60850395 0.19312013 0.5176091  0.76091108 0.88210306]
 [0.60695899 0.17643606 0.46078114 0.73402454 0.83122108]]
1/1 [==============================] - 0s 14ms/step
System operating normally.
[[0.56420311 0.31298734 0.50903793 0.7275655  0.77764766]
 [0.49051507 0.12645161 0.63259267 0.94930477 0.75266331]
 [0.53460453 0.2582544  0.4948611  0.98749702 0.99237288]
 [0.57420224 0.04392614 0.29625528 0.73915168 1.01268429]
 [0.51581136 0.22635664 0.31045909 0.79770536 1.06366025]]
1/1 [==============================] - 0s 15ms/step
System operating normally.
[[0.56703363 0.09423441 0.5311345  0.80100213 1.08545728]
 [0.62691604 0.29499957 0.44617937 0.85992264 0.97644733]
 [0.66271278 0.10579316 0.57013778 0.81653833 0.86699776]
 [0.614

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 14ms/step
System operating normally.
[[0.330497   0.25962729 0.55118621 0.89698963 0.87727884]
 [0.47210736 0.11632318 0.52575777 0.80514235 0.86096508]
 [0.51671527 0.20333119 0.47810979 0.88111222 0.99117232]
 [0.52443274 0.04872243 0.55110603 0.68897734 0.88704758]
 [0.48714397 0.13522282 0.45310755 0.71212912 0.90125411]]
1/1 [==============================] - 0s 14ms/step
System operating normally.
[[ 0.67601496 -0.03914406  0.45786807  0.84573978  0.97953555]
 [ 0.67807669 -0.0272667   0.49237793  0.9070343   0.97948651]
 [ 0.58610143  0.04146757  0.48561906  0.88909257  1.12486545]
 [ 0.66347897  0.0955677   0.41480032  0.72831867  0.90498797]
 [ 0.48675312  0.08596281  0.4523214   0.94097303  0.99801048]]
1/1 [==============================] - 0s 15ms/step
System operating normally.
[[0.60085523 0.07242472 0.52298874 0.5735444  0.95065006]
 [0.71429421 0.10595444 0.75432067 0.82557373 1.01282468]
 [0.49019626 0.11223806 0.39085217 0.778

...

1/1 [==============================] - 0s 14ms/step
System operating normally.
[[ 0.65788602  0.12459812  0.27846737  0.70504569  1.02915812]
 [ 0.7302615  -0.06500888  0.45866159  0.79556564  0.98678384]
 [ 0.85584494  0.22881312  0.45427006  0.77495552  1.09522992]
 [ 0.65009405  0.14750815  0.28511028  0.88442459  0.84835104]
 [ 0.62062622  0.07476177  0.57251249  0.86025565  0.85055011]]
1/1 [==============================] - 0s 14ms/step
System operating normally.
[[ 0.75665143  0.1504342   0.29130212  0.76957008  1.013563  ]
 [ 0.73095469  0.06368151  0.29133164  0.66893296  0.79932222]
 [ 0.63383063 -0.01002418  0.41510348  0.87298617  0.90824148]
 [ 0.66805725  0.29799058  0.44073229  0.71350201  1.0048407 ]
 [ 0.82224666  0.13022034  0.25502375  0.67371039  1.12999223]]
1/1 [==============================] - 0s 17ms/step
System operating normally.
[[ 0.68562326  0.33526034  0.3829702   0.78894828  0.89074386]
 [ 0.56370369  0.15319648  0.27159156  0.89665054  0.86941994]
 [ 0.

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 14ms/step
System operating normally.
[[0.57202466 0.22019283 0.49367487 0.77294705 0.9108258 ]
 [0.63332963 0.29422653 0.24970678 0.5900986  0.84005751]
 [0.67108795 0.21775554 0.27439083 0.6694141  1.01912879]
 [0.59202848 0.12966379 0.03507405 0.93772539 0.95396636]
 [0.76230618 0.2006069  0.17686194 0.91975981 0.87686573]]
1/1 [==============================] - 0s 13ms/step
System operating normally.
[[0.60258045 0.08325407 0.47011053 0.88669839 0.98473572]
 [0.64301911 0.17623925 0.31790751 0.71661893 0.94733542]
 [0.54049784 0.16260366 0.27448341 0.88620832 0.97307009]
 [0.61349663 0.0452281  0.27927564 0.77630784 0.93869867]
 [0.778043   0.06074906 0.51132647 0.94635671 1.19446651]]
1/1 [==============================] - 0s 14ms/step
System operating normally.
[[ 0.71999155  0.05585993  0.33272424  0.79659531  1.09151173]
 [ 0.6322987   0.15150012  0.44002162  0.92737115  1.07338076]
 [ 0.58271229 -0.00391858  0.33705313  0.73544049  0.81

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


KeyboardInterrupt: 